<a href="https://colab.research.google.com/github/alokchoudharyguliya/NLP/blob/main/Neural_Machine_Translation_with_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf

In [1]:
!wget https://www.manythings.org/anki/fra-eng.zip

--2025-08-18 18:23:23--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8143096 (7.8M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.77M  27.1MB/s    in 0.3s    

2025-08-18 18:23:24 (27.1 MB/s) - ‘fra-eng.zip’ saved [8143096/8143096]



In [2]:
!unzip "/content/fra-eng.zip" -d "/content/dataset/"

Archive:  /content/fra-eng.zip
  inflating: /content/dataset/_about.txt  
  inflating: /content/dataset/fra.txt  


In [5]:
text_dataset=tf.data.TextLineDataset("/content/dataset/fra.txt")